## Importing data and required packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings

## Import csv data as pandas Dataframe

In [2]:
df = pd.read_csv('stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## Preparing X and Y variables

In [3]:
X = df.drop(columns='math_score', axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [4]:
y = df['math_score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [5]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OnehotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [6]:
X = preprocessor.fit_transform(X)

In [7]:
X.shape

(1000, 19)

In [8]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

### Creata an evaluate function to give all metrics after model training

In [9]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae,rmse,r2_square

In [10]:
models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighours Regresson" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoosting Regressor" : CatBoostRegressor(verbose=False),
    "AdaBoostRegressor" : AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in  range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance for training set')
    print('-- Root mean square error: {:4f}'.format(model_train_rmse))
    print('-- Mean Absolute error: {:4f}'.format(model_train_mae))
    print('-- R2 score: {:4f}'.format(model_train_r2))

    print('Model Performance for testing set')
    print('-- Root mean square error: {:4f}'.format(model_test_rmse))
    print('-- Mean Absolute error: {:4f}'.format(model_test_mae))
    print('-- R2 score: {:4f}'.format(model_test_r2))


    r2_list.append(model_test_r2)

    print('='*45)
    print('\n')

LinearRegression
Model Performance for training set
-- Root mean square error: 5.323051
-- Mean Absolute error: 4.266712
-- R2 score: 0.874317
Model Performance for testing set
-- Root mean square error: 5.393994
-- Mean Absolute error: 4.214763
-- R2 score: 0.880433


Lasso
Model Performance for training set
-- Root mean square error: 6.593816
-- Mean Absolute error: 5.206303
-- R2 score: 0.807146
Model Performance for testing set
-- Root mean square error: 6.519695
-- Mean Absolute error: 5.157882
-- R2 score: 0.825320


Ridge
Model Performance for training set
-- Root mean square error: 5.323325
-- Mean Absolute error: 4.264988
-- R2 score: 0.874304
Model Performance for testing set
-- Root mean square error: 5.390387
-- Mean Absolute error: 4.211101
-- R2 score: 0.880593


K-Neighours Regresson
Model Performance for training set
-- Root mean square error: 5.705467
-- Mean Absolute error: 4.512250
-- R2 score: 0.855610
Model Performance for testing set
-- Root mean square error: 7.2